# Historical NFL Betting Data Cleaning

Here I will be cleaning the data frame that contains historical NFL betting data going back to the 2020 NFL season.

In [1]:
import numpy as np
import pandas as pd
import re

In [9]:
df = pd.read_csv('/Users/epainter/Desktop/bet_model_v2/data/raw/hist_bet_raw.csv')

Here we are removing rows that contain playoff data. I felt that NFL playoff games do not reflect actual team abilites and are less predictable than regular season games, which is the point of this model. 

In [10]:
df = pd.concat(
    [
        df[~df['Week'].str.contains('Playoffs')].T.shift(1).T.drop(columns=['Round', 'Day', 'Date', 'Time (ET)', 
                                                                            'Week', 'Unnamed: 8']),
        df[~df['Week'].str.contains('Playoffs')]['Week']
    ], 
    axis=1
)


In [11]:
new_col_names = {'Unnamed: 4': 'fav_home', 'Favorite(Seed)': 'favorite', 'Score': 'score', 'Spread': 'spread', 
                 'Underdog(Seed)': 'underdog', 'Over/Under': 'ou', 'Week': 'week' }

df = df.rename(columns=new_col_names)

df

,fav_home,favorite,score,spread,underdog,ou,week
0,@,Kansas City Chiefs,W 34-20,W -9.5,Houston Texans,O 53.5,2020 Regular Season - Week 1
1,@,Atlanta Falcons,L 25-38,L -1,Seattle Seahawks,O 49.5,2020 Regular Season - Week 1
2,@,Baltimore Ravens,W 38-6,W -7,Cleveland Browns,U 47,2020 Regular Season - Week 1
3,@,Buffalo Bills,W 27-17,W -6.5,New York Jets,O 39.5,2020 Regular Season - Week 1
4,NaN,Las Vegas Raiders,W 34-30,W -3,Carolina Panthers,O 48,2020 Regular Season - Week 1
...,...,...,...,...,...,...,...
1105,@,Los Angeles Chargers,L 12-13,L -3.5,Kansas City Chiefs,U 35,2023 Regular Season - Week 18
1106,NaN,Philadelphia Eagles,L 10-27,L -5,New York Giants,U 42.5,2023 Regular Season - Week 18
1107,@,San Francisco 49ers,L 20-21,L -5.5,Los Angeles Rams,O 40,2023 Regular Season - Week 18
1108,NaN,Dallas Cowboys,W 38-10,W -13,Washington Commanders,O 47.5,2023 Regular Season - Week 18


As we can see, the values in this data frame are a mess. Below is how I decided to clean it up.

In [12]:
# Replace '@' with 1 and NaN with 0 in the fav_home column
df['fav_home'] = df['fav_home'].replace({'@': 1, 'N': 0, np.nan: 0})
df['fav_home'] = df['fav_home'].astype(int)

# Split the score column into fav_ml_result, fav_score, and und_score, accounting for optional overtime
df[['fav_ml_result', 'fav_score', 'und_score']] = df['score'].str.extract(r'([WLT]) (\d+)-(\d+)(?: \(OT\))?')
df['fav_ml_result'] = df['fav_ml_result'].map({'W': 1, 'L': 0, 'T': 0})
df['fav_score'] = df['fav_score'].fillna(0).astype(int)
df['und_score'] = df['und_score'].fillna(0).astype(int)

# Split the spread column into fav_sp_result and spread
df[['fav_sp_result', 'spread']] = df['spread'].str.extract(r'([WLP]) ([-+]?\d*\.?\d+)')
df['fav_sp_result'] = df['fav_sp_result'].map({'W': 1, 'L': 0, 'P': 0})
df['spread'] = df['spread'].astype(float)

# Split the ou column into ou_result and ou_value
df[['ou_result', 'ou_value']] = df['ou'].str.extract(r'([OUP]) (\d*\.?\d+)')
df['ou_result'] = df['ou_result'].map({'O': 1, 'U': 0, 'P': 0})
df['ou_value'] = df['ou_value'].astype(float)

# Split the week column into season and week
df[['season', 'week']] = df['week'].str.extract(r'(\d{4}) Regular Season - Week (\d+)')
df['season'] = df['season'].astype(int)
df['week'] = df['week'].astype(int)

# Dropping unnecessary columns
col_drop = ['score', 'ou']

df = df.drop(columns=col_drop)

# Reorganize columns
df = df[['favorite', 'underdog', 'fav_home', 'fav_score', 'und_score', 'spread', 'ou_value', 'fav_ml_result', 
         'fav_sp_result', 'ou_result', 'week', 'season']]

df

/var/folders/4q/c8tkm_cj1f16_m9nvk_zn4100000gn/T/ipykernel_11004/1489097788.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fav_home'] = df['fav_home'].replace({'@': 1, 'N': 0, np.nan: 0})


,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
1105,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023
1106,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023
1107,San Francisco 49ers,Los Angeles Rams,1,20,21,-5.5,40.0,0,0.0,1,18,2023
1108,Dallas Cowboys,Washington Commanders,0,38,10,-13.0,47.5,1,1.0,1,18,2023


Next step is to make sure the team names are correct so that we can merge our data later.

For example, in most recent years Washington has gone by the *Washington Commanders*, where in 2020 their team name was the *Washington Football Team*

In [23]:
df['favorite'] = df['favorite'].replace({
    'Washington Football Team': 'Washington Commanders'
})

df['underdog'] = df['underdog'].replace({
    'Washington Football Team': 'Washington Commanders'
})


In [13]:
df.isna().sum()

favorite         0
underdog         0
fav_home         0
fav_score        0
und_score        0
spread           2
ou_value         0
fav_ml_result    0
fav_sp_result    2
ou_result        0
week             0
season           0
dtype: int64

We can see that there are two *Nan* values in our data frame. I will impute them with the correct spread and result.

In [24]:
df[df['spread'].isna()]

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season
448,Washington Commanders,Seattle Seahawks,1,17,15,NaN,47.0,1,NaN,0,12,2021
506,Las Vegas Raiders,Denver Broncos,1,17,13,NaN,41.0,1,NaN,0,16,2021


In [33]:
df.loc[448, 'spread'] = 1.0
df.loc[448, 'fav_sp_result'] = 1

df.loc[506, 'spread'] = 1.0
df.loc[506, 'fav_sp_result'] = 1

# Write to CSV

In [39]:
fp = '/Users/epainter/Desktop/bet_model_v2/data/clean/hist_bet_clean.csv'

df.to_csv(fp, index=False)

print(f"Data saved to {fp}")

Data saved to /Users/epainter/Desktop/bet_model_v2/data/clean/hist_bet_clean.csv
